In [1]:
import re
import math
from copy import deepcopy
from ctypes import *
import numpy as np

class ts_sensor_data_t():
    def __init__(self, TS_SCAN_SIZE):
        self.TS_SCAN_SIZE = TS_SCAN_SIZE
        self.timestamp = 0
        self.q1 = 0
        self.q2 = 0
        self.scan = []

        for i in range(0, self.TS_SCAN_SIZE):
            elements = [0]*4
            self.scan.append(elements)

class ExtendedFunctions():
    def __init__(self):
        self.TEST_SCAN_SIZE = 682
        self.TEST_MIN_DIST  = 20
        self.TEST_ANGLE_MIN = -180
        self.TEST_ANGLE_MAX = +180
        self.TEST_OFFSET_LASER = 145
        self.TEST_HOLE_WIDTH = 600
        self.M_PI = 3.14159265358979323846

        self.TS_SCAN_SIZE = 683
        self.TS_MAP_SIZE = 360
        self.TS_MAP_SCALE = 0.017578125
        self.TS_DISTANCE_NO_DETECTION = 4000
        self.TS_NO_OBSTACLE = 65500
        self.TS_OBSTACLE = 0
        self.TS_HOLE_WIDTH = 600
        
        self.val = 1
        self.RAND_MAX = 0x7fff
        
        self.my_so = '/home/xilinx/LinLib/SharedLibrary/libLiDAR.so'
        self.myLiDAR = cdll.LoadLibrary(self.my_so)
        my_string = "/dev/ttyUSB0"
        self.port = c_char_p(my_string.encode('utf-8'))

    #Start the Sensor
    def Start_Sensor(self):
        mp = (c_float * self.TEST_SCAN_SIZE)()
        select = c_int(0)
        if (self.myLiDAR.main(mp, self.port, select) == 1):
            return True
        else:
            return False

    #Stop the Sensor
    def Stop_Sensor(self):
        mp = (c_float * self.TEST_SCAN_SIZE)()
        select = c_int(2)
        self.myLiDAR.main(mp, self.port, select)

    #Read Scans from sensor
    def Read_Sensor(self, scan):
        mp = (c_float * self.TEST_SCAN_SIZE)()
        select = c_int(1)
        self.myLiDAR.main(mp, self.port, select)

        nb_points = 0
        SPAN = 1
        for i in range(0, self.TEST_SCAN_SIZE):
            for j in range(0, SPAN):
                angle_deg = self.TEST_ANGLE_MIN + float((i * SPAN + j)) * (self.TEST_ANGLE_MAX - self.TEST_ANGLE_MIN) / (self.TEST_SCAN_SIZE * SPAN - 1)
                angle_rad = angle_deg * self.M_PI / 180
                if ((i > 45) and (i < self.TEST_SCAN_SIZE - 45)):
                    if (mp[i] == 0):
                        scan[nb_points][1] = self.TS_DISTANCE_NO_DETECTION * math.cos(angle_rad)
                        scan[nb_points][2] = self.TS_DISTANCE_NO_DETECTION * math.sin(angle_rad)
                        scan[nb_points][3] = self.TS_NO_OBSTACLE
                        scan[nb_points][1] += self.TEST_OFFSET_LASER
                        nb_points += 1
                    if (mp[i] > (self.TEST_HOLE_WIDTH / 2)):
                        scan[nb_points][1] = mp[i] * math.cos(angle_rad)
                        scan[nb_points][2] = mp[i] * math.sin(angle_rad)
                        scan[nb_points][3] = self.TS_OBSTACLE
                        scan[nb_points][1] += self.TEST_OFFSET_LASER
                        nb_points += 1

        for i in range(0, self.TS_SCAN_SIZE):
            scan[i][0] = nb_points

        return scan


    #Calculate New Map by updating data points
    def draw_scan(self, scan, dmap, dpos):
        c = math.cos(dpos[2] * self.M_PI / 180)
        s = math.sin(dpos[2] * self. M_PI / 180)
        x1 = math.floor(dpos[0] * self.TS_MAP_SCALE + 0.5)
        y1 = math.floor(dpos[1] * self.TS_MAP_SCALE + 0.5)

        for i in range(0, int(scan[0][0])):
            xs = scan[i][1]
            ys = scan[i][2]
            vs = scan[i][3]

            if (vs != self.TS_NO_OBSTACLE):
                x2p = c * xs - s * ys
                y2p = s * xs + c * ys
                x2p *= self.TS_MAP_SCALE
                y2p *= self.TS_MAP_SCALE
                x2 = int(math.floor(dpos[0] * self.TS_MAP_SCALE + x2p + 0.5))
                y2 = int(math.floor(dpos[1] * self.TS_MAP_SCALE + y2p + 0.5))
                if ((x2 >= 0) and (y2 >= 0) and (x2 < self.TS_MAP_SIZE) and (y2 < self.TS_MAP_SIZE)):
                    dmap[y2 * self.TS_MAP_SIZE + x2] = 0


    def record_map(self, imap, overlay, filename, width, height):
        output2 = open(filename, 'w')

        output2.write('P2\n%d %d 255\n' % (width, height))
        ry = (self.TS_MAP_SIZE - height) / 2

        for yp in range(0, height):
            rx = (self.TS_MAP_SIZE - width) / 2
            for xp in range(0, width):
                if (overlay[int((self.TS_MAP_SIZE - 1 - ry) * self.TS_MAP_SIZE + rx)] == 0):
                    output2.write('0 ')
                else:
                    output2.write("%d " % (int((imap[int((self.TS_MAP_SIZE - 1 - ry) * self.TS_MAP_SIZE + rx)]) >> 8)))

                rx += 1
            output2.write('\n')
            ry += 1

        output2.close()

In [2]:
from pynq import Xlnk
import numpy as np
import pynq.lib.dma
from pynq import Overlay
import random
import struct
import time

tinySLAM = Overlay('/home/xilinx/MyFiles/tinySLAM_Switch/V3/design_1.bit')
tiny_ip = tinySLAM.HLS_tinySLAM_0
dma_run = tinySLAM.dma_run
dma_pos = tinySLAM.dma_pos

TS_SCAN_SIZE = 683
TS_MAP_SIZE = 360
RAND_MAX = 0x7fff
TS_MAP_SCALE = 0.017578125

map_start = 0x40000
scan_start = 0x04000
in_value = 0x80000
out_value = 0x80008

class HLS_Functions():
    
    #//////////////////////////////////////////////////// Bit Converters ////////////////////////////////////////////////
    def floatToBits(self, f):
        s = struct.pack('>f', f)
        return struct.unpack('>l', s)[0]

    def BitsTofloat(self, i):
        s = struct.pack('>l', i)
        return struct.unpack('>f', s)[0]
    
    #//////////////////////////////////////////////////// MAP INIT /////////////////////////////////////////////////////
    def HLS_map_init(self):
        #Initialize Variables
        Run = xlnk.cma_array(shape=(1,), dtype=np.uint32)
        map1 = []
            
        #print("Activate Map Init...")
        Run[0] = 1
        dma_run.sendchannel.transfer(Run)
        dma_run.sendchannel.wait()
        
        #Delay to Allow time for calculation to complete
        time.sleep(0.1)
        
        #print("Reading Map...")
        for v in range(0, 64800):

            #Read in the 32 bit value
            location = map_start + 4*v
            val = tiny_ip.read(location)

            #Split into two 16 bit values
            upper = (val >> 16) & 0xFFFF
            lower = val & 0xFFFF

            #Append
            map1.append(lower)
            map1.append(upper)
            
        #print(map1)
        return map1
    
    #////////////////////////////////////////////// MONTE CARLO + MAP UPDATE //////////////////////////////////////////////////
    def HLS_Main(self, scan, mp, pos):
        #Initialize Variables
        Run = xlnk.cma_array(shape=(1,), dtype=np.uint32)
        pos2 = xlnk.cma_array(shape=(3,), dtype=np.float32)
        map1 = []
        
        x = 0
        #print("Writing Scan...")
        for i in range(0, TS_SCAN_SIZE):
            for j in range(0, 4):
                location = scan_start + 4*x
                val = scan[i][j]
                var = self.floatToBits(val)
                tiny_ip.write(location,var)
                x += 1
        
        #print("Writing Map...")
        for v in range(0, 64800):
            #Combine two 16 bit values into a Single Value
            location = map_start + 4*v
            #val = (mp[2*v] << 16) | mp[(2*v)+1]
            val = (mp[(2*v)+1] << 16) | mp[2*v]

            #Write the 32 bit value
            tiny_ip.write(location,val)

        #print("Streaming Position")
        dma_pos.sendchannel.transfer(pos)
        dma_pos.sendchannel.wait()
        
        #print("Activate Main...")
        Run[0] = 2
        dma_run.sendchannel.transfer(Run)
        dma_run.sendchannel.wait()
        
        #print("Recieve New Position")
        dma_pos.recvchannel.transfer(pos2)
        dma_pos.recvchannel.wait()
        
        #Delay to Allow time for calculation to complete
        time.sleep(0.1)
        
        #print("Reading Map...")
        for v in range(0, 64800):

            #Read in the 32 bit value
            location = map_start + 4*v
            val = tiny_ip.read(location)

            #Split into two 16 bit values
            upper = (val >> 16) & 0xFFFF
            lower = val & 0xFFFF

            #Append
            map1.append(lower)
            map1.append(upper)
        
        return map1, pos2

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import cv2
from pynq import Xlnk
import numpy as np
import pynq.lib.dma
from pynq import Overlay
import random
import datetime

################################################## MAIN ################################################################
#//Initialise Variables
time_start = datetime.datetime.now()
xlnk = Xlnk()
functions = ExtendedFunctions()
hls_func = HLS_Functions()
Sensor_Data = []
told = 0
filename = ""

#Initialize Arrays
scaner = []
sc = []
pos = xlnk.cma_array(shape=(3,), dtype=np.float32)

for i in range(0, 600):
    Sensor_Data.append(ts_sensor_data_t(functions.TS_SCAN_SIZE))
        
for i in range(0, functions.TS_SCAN_SIZE):
    elements = [0]*4
    scaner.append(elements)
    sc.append(elements)

#////////////////////////////////////Initial machine pose//////////////////////////////////////
pos[0] = (functions.TS_MAP_SIZE / functions.TS_MAP_SCALE)*0.3
pos[1] = (functions.TS_MAP_SIZE / functions.TS_MAP_SCALE)*0.3
pos[2] = 0.0

print("Map Init.....")
mp = hls_func.HLS_map_init()
traj = hls_func.HLS_map_init()

output = open("test_trajectory.dat", 'w')

go = True
if(functions.Start_Sensor() == True):
    try:
        cnt_scans = 0
        while(go == True):
            sc = functions.Read_Sensor(sc)
            for i in range(0, functions.TS_SCAN_SIZE):
                scaner[i][0] = sc[i][0]
                scaner[i][1] = sc[i][1]
                scaner[i][2] = sc[i][2]
                scaner[i][3] = sc[i][3]

            timestamp = Sensor_Data[cnt_scans].timestamp

            #/////////////////////////// HLS Functions ///////////////////////////////
            mp, pos2 = hls_func.HLS_Main(scaner, mp, pos)

            for i in range(0, 3):
                pos[i] = pos2[i]

            print("#%d : %lg %lg %lg" % (cnt_scans, pos[0], pos[1], pos[2]))

            #///////////////////////////Set Current Pose as Obstacle Point//////////////////////////
            x = (int(math.floor(pos[0] * functions.TS_MAP_SCALE + 0.5)))
            y = (int(math.floor(pos[1] * functions.TS_MAP_SCALE + 0.5)))

            if ((x >= 0) and (y >= 0) and (x < functions.TS_MAP_SIZE) and (y < functions.TS_MAP_SIZE)):
                traj[y * functions.TS_MAP_SIZE + x] = 0

            #///////////////////////////Update the obstacle point map///////////////////////////////
            functions.draw_scan(scaner, traj, pos)

            if(cnt_scans % 10 == 0):
                filename = "/home/xilinx/DataFiles/test_lab_reversed2.pgm"
                functions.record_map(mp, traj, filename, functions.TS_MAP_SIZE, functions.TS_MAP_SIZE)

                image = cv2.imread(filename)
                cv2.circle(image, (x, functions.TS_MAP_SIZE - y), 5, (0, 0, 255), 3);
                plt.imshow(image)
                plt.show()

            #//Set values equal to current for next loop.
            told = timestamp
            output.write("\n")
            cnt_scans += 1
            
            #Exit While Loop on Ctrl-C
    except KeyboardInterrupt:
            go = False

    output.close()
    functions.Stop_Sensor()

    #// Record the map
    print("Converting to PGM")
    filename = "/home/xilinx/DataFiles/test_lab_reversed2.pgm"
    functions.record_map(mp, traj, filename, functions.TS_MAP_SIZE, functions.TS_MAP_SIZE)

    image = cv2.imread(filename)
    cv2.circle(image, (x, functions.TS_MAP_SIZE - y), 5, (0, 0, 255), 3);
    plt.imshow(image)
    plt.show()

    print("Finished!!")
    time_end = datetime.datetime.now()
    delta = time_end - time_start

    print("Time:", delta)
else:
    print("Error in Sensor Setup, Closing System!!!")